## Headers

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from string import punctuation
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer=nltk.stem.WordNetLemmatizer()
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cosine
import time
import datetime

In [2]:
 nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AbhiG\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Loading Dataset

In [3]:
Dp=pd.read_csv("I:/FLAIRMINDS/Product_matching/product-data-category.csv")
Dp

,id,product_id,website_id,normalized_leaf_category_id,department_name,iksula_category_identifier,product_url,product_title,model_number,manufacturing_brand
0,1311275,48171894,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B09KQBQLZQ,Multifunctional Air Compressor,NaN,Generic
1,1310580,43184194,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B06Y3W2676,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",312-071B,Beverage Air
2,1310903,46036109,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B08NK54DG2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,NaN,Vbestlife
3,1311137,47687124,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B08FXSQTCB,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,NaN,RYOBI
4,1311979,52530864,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B08P5KFNRD,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,NaN,Fafeicy
...,...,...,...,...,...,...,...,...,...,...
4022,1311446,48873158,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B01L8Q30SI,Professional Portable Airbrush Black Mini Air ...,‎31CPR010,Generic
4023,1310830,45490321,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B077XB9745,Update -3 Set Pressure Electric High Pressure ...,NaN,TECHTONGDA
4024,1311723,49827612,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B09D7HK835,20V Tire Inflator Air Compressor Portable Bund...,NaN,WORKSITE
4025,1311219,47891808,2,19476,tools,ik_tools_portable air compressors,https://www.amazon.com/dp/B09DCK5H1M,HPDAVV High Pressure Air Compressor Oil-Free 1...,NaN,HPDAVV


## product_title dataframe

In [6]:
title=pd.DataFrame(Dp["product_title"])

In [7]:
title.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4027 entries, 0 to 4026
Data columns (total 1 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   product_title  4026 non-null   object
dtypes: object(1)
memory usage: 31.6+ KB


In [8]:
title.isnull().values.any()

True

In [9]:
print(title.isnull().sum().sum())
title=title.dropna()

1


In [10]:
## checking if stopwords are available or not

stopWords = set(stopwords.words('english'))
def remove_stopwords(text):
    words = [word for word in text.split() if word.lower() in stopWords]
    if not words:
        return "Null"
    return " ".join(words)

In [11]:
title["stopwords"]=title.apply(lambda row: remove_stopwords(row["product_title"]),axis=1)

In [12]:
title

,product_title,stopwords
0,Multifunctional Air Compressor,Null
1,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",Null
2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,for for
3,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,with
4,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,with
...,...,...
4022,Professional Portable Airbrush Black Mini Air ...,for
4023,Update -3 Set Pressure Electric High Pressure ...,Null
4024,20V Tire Inflator Air Compressor Portable Bund...,with
4025,HPDAVV High Pressure Air Compressor Oil-Free 1...,Null


## Cleaning all titles

In [13]:
def cleaning(text):
    text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)  ## http tags removal
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'[\.;:,\?\"\'\/-]','',text)                            ## punctiations removal
    text=re.sub(r' {1,}',' ',text)                                        ## extra space removal
    words = []
    for word in text.split():
        word=word.lower()
        if word not in stopWords:                                        ## stopwords removal
            if word not in punctuation:
                word=nltk.stem.WordNetLemmatizer().lemmatize(word)       ## lemmatization

            words.append(word)
    return " ".join(words)

In [14]:
x="i am planning to go to hill station believes"
cleaning(x)

'planning go hill station belief'

In [15]:
## appending cleaned title to title dataframe

title["cleaned title"]=title.apply(lambda row: cleaning(row["product_title"]),axis=1)

In [16]:
title

,product_title,stopwords,cleaned title
0,Multifunctional Air Compressor,Null,multifunctional air compressor
1,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",Null,beverage air 312071b compressor 115v 60 hz r13...
2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,for for,vbestlife air compressor spray gun， airbrush， ...
3,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,with,ryobi p737d 18volt one+ cordless high pressure...
4,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,with,2pcs air compressor cylinder head small 253p a...
...,...,...,...
4022,Professional Portable Airbrush Black Mini Air ...,for,professional portable airbrush black mini air ...
4023,Update -3 Set Pressure Electric High Pressure ...,Null,update 3 set pressure electric high pressure a...
4024,20V Tire Inflator Air Compressor Portable Bund...,with,20v tire inflator air compressor portable bund...
4025,HPDAVV High Pressure Air Compressor Oil-Free 1...,Null,hpdavv high pressure air compressor oilfree 15...


In [17]:
n_product=title.shape[0]
print(n_product)
title

4026


,product_title,stopwords,cleaned title
0,Multifunctional Air Compressor,Null,multifunctional air compressor
1,"Beverage Air 312-071B Compressor, 115V, 60 Hz,...",Null,beverage air 312071b compressor 115v 60 hz r13...
2,Vbestlife Air Compressor Spray Gun， Airbrush， ...,for for,vbestlife air compressor spray gun， airbrush， ...
3,Ryobi P737D 18-Volt ONE+ Cordless High Pressur...,with,ryobi p737d 18volt one+ cordless high pressure...
4,2Pcs Air Compressor Cylinder Head Small 2.5/3P...,with,2pcs air compressor cylinder head small 253p a...
...,...,...,...
4022,Professional Portable Airbrush Black Mini Air ...,for,professional portable airbrush black mini air ...
4023,Update -3 Set Pressure Electric High Pressure ...,Null,update 3 set pressure electric high pressure a...
4024,20V Tire Inflator Air Compressor Portable Bund...,with,20v tire inflator air compressor portable bund...
4025,HPDAVV High Pressure Air Compressor Oil-Free 1...,Null,hpdavv high pressure air compressor oilfree 15...


## Sentence-Transformers library

In [18]:
pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.


In [19]:
from sentence_transformers import SentenceTransformer
model_name='bert-base-nli-mean-tokens'                    ## BERT model
model=SentenceTransformer(model_name)

In [20]:
s=title["cleaned title"].iloc[1]
s

'beverage air 312071b compressor 115v 60 hz r134a 240'

In [21]:
vec=model.encode(s)

In [22]:
dimensions=len(vec)
dimensions

768

In [23]:
vec1=model.encode(title["cleaned title"].iloc[2])

In [24]:
vec1

array([ 5.64172328e-01,  1.08898580e+00,  9.64886189e-01,  2.95702100e-01,
       -1.25234127e-01, -8.52528811e-01,  2.31795564e-01, -1.76708609e-01,
        6.09552860e-01, -4.36044455e-01, -4.44099635e-01, -3.41639221e-01,
        9.76449370e-01,  6.49099827e-01,  2.88555682e-01,  1.69631839e-01,
       -9.14415181e-01,  6.71862423e-01,  9.87214446e-01, -3.49643797e-01,
       -6.22138441e-01, -2.26730496e-01,  1.72552288e-01, -3.26403022e-01,
        4.95819449e-01,  7.20625699e-01,  1.04987666e-01,  2.70726293e-01,
       -4.50515091e-01, -1.10936880e-01, -6.07368588e-01,  8.20974827e-01,
        4.85259265e-01, -1.34145069e+00,  4.55497235e-01,  1.47108734e-01,
       -2.45624945e-01,  8.91204655e-01,  1.25196248e-01, -1.06296909e+00,
        8.47432837e-02, -3.47486824e-01,  8.87308836e-01, -1.04716182e+00,
       -1.26266479e+00, -5.20219207e-01, -3.06000561e-02,  8.72573674e-01,
       -2.90032774e-01, -7.48419046e-01,  7.15078771e-01, -2.62004524e-01,
        1.32369328e+00, -

In [25]:
x=[1,2,3,4]
y=[5,2,4,4]
cosine(x,y)             #1- (u.v)/(||u||*||v||)

0.13507952290745884

In [26]:
cosine(vec,vec1)

0.4048285484313965

In [27]:
cosine(vec,vec)              ##cosine(vec,vec) = distance score = 1-similarity score

0

### eucledian distance = sqrt(2*(1-cosine_similarity))

### cosine(vec,vec) = distance score = 1-similarity score

In [28]:
title["cleaned title"][1]

'beverage air 312071b compressor 115v 60 hz r134a 240'

## vector formation (word embeddings)

In [31]:
score=[]
start=time.time()
for i in range(n_product):
    score.append(model.encode(title["cleaned title"].iloc[i]))
score=np.array(score)
print(score.shape)
end=time.time()
convert = str(datetime.timedelta(seconds = end-start))
print("Execution time: ",convert,"hrs: min: sec")

(4026, 768)
Execution time:  0:02:05.393626 hrs: min: sec


## cosine similarity calculations

In [32]:
N=np.zeros(( n_product, n_product))
start=time.time()
for i in range( n_product):
    for j in range( n_product):
        N[i][j]=1-cosine(score[i,:],score[j,:])
end=time.time()
convert = str(datetime.timedelta(seconds = end-start))
print("Execution time: ",convert,"hrs: min: sec")

Execution time:  0:19:27.687811 hrs: min: sec


In [33]:
N

array([[1.        , 0.63900614, 0.53383845, ..., 0.52500993, 0.53878069,
        0.52826506],
       [0.63900614, 1.        , 0.59517145, ..., 0.560498  , 0.71302372,
        0.66694492],
       [0.53383845, 0.59517145, 1.        , ..., 0.57800245, 0.55739003,
        0.60499054],
       ...,
       [0.52500993, 0.560498  , 0.57800245, ..., 1.        , 0.64474195,
        0.58676076],
       [0.53878069, 0.71302372, 0.55739003, ..., 0.64474195, 1.        ,
        0.82256573],
       [0.52826506, 0.66694492, 0.60499054, ..., 0.58676076, 0.82256573,
        1.        ]])

## saving scores into dataframe

In [34]:
similarity=pd.DataFrame(N)
similarity.columns=title["product_title"].iloc[0:n_product]
similarity.index=title["product_title"].iloc[0:n_product]
similarity

product_title,Multifunctional Air Compressor,"Beverage Air 312-071B Compressor, 115V, 60 Hz, R134A 240",Vbestlife Air Compressor Spray Gun， Airbrush， Mini Air Compressor for Tattoo for Cake,Ryobi P737D 18-Volt ONE+ Cordless High Pressure Inflator with Digital Gauge & 18-Volt ONE+ Lithium-Ion 1.5 Ah LITHIUM+ Compact Battery (Bulk Packaged),"2Pcs Air Compressor Cylinder Head Small 2.5/3P Aluminum Cooling Parts 48mm x 62mm, Pneumatic Driving, with 2 Cylinder Heads","TBVECHI High Pressure Air Compressor Pump, 300BAR 4500PSI Electric Air Pump Airgun Scuba Rifle PCP 110V","Air pump XMYYHH Electric Bike Pump,Air Compressor,Portable Compressor Digital Car Tyre Pump,Rechargeable with LCD Display,for Car Bicycle Tires Balls Kayaking (Color : Black)","Modengzhe Portable Car Tire Inflator with Presure Gauge, DC 12V Car Cigarette Lighter Powered Mini Air Compressor Pump for Vehicles Motorcycles Bicycles Mopeds Inflatable Boats or Balls","Flexzilla Portable Air Compressor with Quiet Technology, 1 HP, 5 Gallon, Industrial Grade Pump",Campbell Hausfeld Campbell Hausfeld 2 Gal. Twin-Stack Air Compressor Kit,...,"425C 175 Psi Platinum Compressor Kit (Gen 2) W/Matte Black Filter, 1/4"" Port (12V, 33% Duty @ 175 Psi)",4 pics Air Compressor Belts PJ373 for H1504ST A700062 Husky Pumps,"beduan Brass Tire Inflator Lock on Air Chuck 1/4"" Hose Barb, Portable Compressor Tire inflator Valve Chuck Hose (Pack of 2)",760504007 Switch Replacement for Homelite OEM,Mini Portable 150PSI Car Tyre Air Compressor Pump 12V Electric Tire Inflator,Professional Portable Airbrush Black Mini Air Compressor Fashionable Design w/Built-in Holder for Makeup Tattoo Ink Hobby Decorating Commercial Arts,Update -3 Set Pressure Electric High Pressure Air Compressor Pump Machine- Airgun 30Mpa,20V Tire Inflator Air Compressor Portable Bundle with 20V Cordless Drill/Driver Kit,HPDAVV High Pressure Air Compressor Oil-Free 1.5 KW / 2 HP - 110v / 60Hz - 2.1cfm @ 4500 Psi - Scuba Tank/PCP Rife/Paintball Air Gun Filling Station Air Breathed Directly,CNCEST PCP Air Compressor 30MPA High Pressure Air Pump for Diving Pump Inflator Paintball Scuba Tank Filling Pump
product_title,,,,,,,,,,,,,,,,,,,,,
Multifunctional Air Compressor,1.000000,0.639006,0.533838,0.383890,0.648692,0.495935,0.387653,0.393582,0.518613,0.749233,...,0.365408,0.612805,0.534610,0.334135,0.529258,0.403895,0.586038,0.525010,0.538781,0.528265
"Beverage Air 312-071B Compressor, 115V, 60 Hz, R134A 240",0.639006,1.000000,0.595171,0.469983,0.635545,0.714474,0.505343,0.525744,0.568680,0.744082,...,0.622135,0.776473,0.620597,0.507344,0.663225,0.467077,0.710244,0.560498,0.713024,0.666945
Vbestlife Air Compressor Spray Gun， Airbrush， Mini Air Compressor for Tattoo for Cake,0.533838,0.595171,1.000000,0.426371,0.561932,0.551016,0.514239,0.549826,0.515907,0.661068,...,0.368761,0.609605,0.564948,0.438601,0.560302,0.626823,0.575419,0.578002,0.557390,0.604991
Ryobi P737D 18-Volt ONE+ Cordless High Pressure Inflator with Digital Gauge & 18-Volt ONE+ Lithium-Ion 1.5 Ah LITHIUM+ Compact Battery (Bulk Packaged),0.383890,0.469983,0.426371,1.000000,0.621419,0.613747,0.574869,0.568068,0.675214,0.519323,...,0.656976,0.527904,0.604208,0.382107,0.711985,0.406770,0.551935,0.623110,0.584128,0.421633
"2Pcs Air Compressor Cylinder Head Small 2.5/3P Aluminum Cooling Parts 48mm x 62mm, Pneumatic Driving, with 2 Cylinder Heads",0.648692,0.635545,0.561932,0.621419,1.000000,0.639940,0.576889,0.622749,0.738159,0.752806,...,0.610800,0.671068,0.690958,0.311316,0.736285,0.522920,0.629680,0.710014,0.651045,0.573635
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Professional Portable Airbrush Black Mini Air Compressor Fashionable Design w/Built-in Holder for Makeup Tattoo Ink Hobby Decorating Commercial Arts,0.403895,0.467077,0.626823,0.406770,0.522920,0.463973,0.639388,0.504850,0.506244,0.467727,...,0.510323,0.450421,0.498282,0.319036,0.515093,1.000000,0.396473,0.489616,0.482592,0.408875
Update -3 Set Pressure Electric

In [ ]:
similarity=pd.read_csv("I:/FLAIRMINDS/Product_matching/similarity_values.csv")
similarity

## Top 10 similar product titles

In [35]:
def top_10__similar(p):
    x=similarity.iloc[p].values
    y=[]
    for j in range(n_product):
        y.append([x[j],j])
    y=y[1:]
    print(y[0:5])
    y.sort(key=lambda z:z[0],reverse=True)
    print("Query_Product_Title: ")
    print(Dp["product_url"][p],"-->",title["product_title"][p])
    print("*********************************************\n")
    print("Top 10 similar product title with similarity score are: \n")

    for i in range(10):
        print( f"{y[i][0]:.4f} --> ",Dp["product_url"][y[i][1]],"-->", title["product_title"][y[i][1]])

In [38]:
top_10__similar(55)

[[0.7134063839912415, 1], [0.6904270052909851, 2], [0.5859700441360474, 3], [0.6457716822624207, 4], [0.6773176789283752, 5]]
Query_Product_Title: 
https://www.walmart.com/ip/Porter-Cable-PCFP3KIT-3-Piece-Nailer-and-0-8-HP-6-Gallon-Oil-Free-Pancake-Air-Compressor-Combo-Kit/910355347 --> Porter-Cable PCFP3KIT 3-Piece Nailer and 0.8 HP 6 Gallon Oil-Free Pancake Air Compressor Combo Kit
*********************************************

Top 10 similar product title with similarity score are: 

1.0000 -->  https://www.walmart.com/ip/Porter-Cable-PCFP3KIT-3-Piece-Nailer-and-0-8-HP-6-Gallon-Oil-Free-Pancake-Air-Compressor-Combo-Kit/910355347 --> Porter-Cable PCFP3KIT 3-Piece Nailer and 0.8 HP 6 Gallon Oil-Free Pancake Air Compressor Combo Kit
0.8944 -->  https://www.walmart.com/ip/Pulsar-3-Gallon-Oil-Free-Pancake-Air-Compressor-with-Air-Hose-and-6-Piece-Accessory-Kit/519058324 --> Pulsar 3 Gallon Oil-Free Pancake Air Compressor with Air Hose and 6 Piece Accessory Kit
0.8915 -->  https://www.walm